## PDI

Aluno: Felipe Vasconcelos

Matrícula: 20110898

A função se chama bottle_detection!

In [12]:
import cv2 as cv
import numpy as np
import imutils


def cap_missing(image):

    (thresh, image) = cv.threshold(image, 128,
                                   255, cv.THRESH_BINARY | cv.THRESH_OTSU)

    image = image[0:100, 110:230]

    total_pixels = image.size

    black_pixels = total_pixels - cv.countNonZero(image)
    qnt_black = (black_pixels / total_pixels) * 100

    if qnt_black > 20:
        return 0
    else:
        return 1


def label(img):
    (thresh, image) = cv.threshold(img, 128,
                                   255, cv.THRESH_BINARY | cv.THRESH_OTSU)

    image = image[150:, 100:260]

    total_pixels = image.size

    qnt_pixels = (cv.countNonZero(image) / total_pixels) * 100

    if qnt_pixels > 51:
        return 1
    elif qnt_pixels < 5:
        return None
    else:
        return 0


def label_align(image):
    image = image[160:260, 100:260]
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    thresh, img = cv.threshold(gray, 150, 255, cv.THRESH_BINARY)

    lines = cv.HoughLines(img, 1, np.pi / 2, threshold=100)

    if lines is not None:
        return 0
    else:
        return 1


def level(image):
    image = image[30:190, 110:240]
    height, width, _ = image.shape
    new_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    (tresh, img) = cv.threshold(
        new_img, 55, 255, cv.THRESH_BINARY_INV)
    contours = cv.findContours(
        img.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    areas = [cv.contourArea(contour) for contour in contours]

    if len(contours) != 0:
        (contours, areas) = zip(
            *sorted(zip(contours, areas), key=lambda a: a[1]))

        (x, y, w, h) = cv.boundingRect(contours[-1])
        aspectRatio = round(h / float(height), 1)
        if aspectRatio > 0.3:
            return 'high'
        elif aspectRatio == 0.3:
            return 'normal'

    return 'low'


def level_to_string(img):

    res = level(img)

    if level(img) == 'low':
        return 0, 1
    elif level(img) == 'normal':
        return 0, 0
    else:
        return 1, 0


def bottle_missing(img):
    (thresh, image) = cv.threshold(img, 128,
                                   255, cv.THRESH_BINARY | cv.THRESH_OTSU)

    image = image[:, 120:270]

    total_pixels = image.size
    black_pixels = total_pixels - cv.countNonZero(image)
    qnt_black = (black_pixels / total_pixels) * 100

    if qnt_black > 30:
        return 0
    else:
        if cap_missing(img) == 0:
            return 0
        return 1


def bottle_detection(img_cru):
    dicionario = {
        "CONTENT_HIGH": 0,
        "CONTENT_LOW": 0,
        "COVER_NONE": 0,
        "BOTTLE_SMASHED": 0,
        "LABEL_WHITE": 0,
        "LABEL_MISPLACED": 0,
        "LABEL_NONE": 0,
        "BOTTLE_NONE": 0
    }

    img = cv.cvtColor(img_cru, cv.COLOR_BGR2GRAY)

    dicionario["BOTTLE_NONE"] = bottle_missing(img)

    if dicionario["BOTTLE_NONE"] == 1:
        pass
    else:
        dicionario["COVER_NONE"] = cap_missing(img)
        dicionario["CONTENT_HIGH"], dicionario["CONTENT_LOW"] = level_to_string(
            img_cru)

        label_test = label(img)

        if label_test == None:
            dicionario['LABEL_NONE'] = 1
        elif label_test == 1:
            dicionario['LABEL_WHITE'] = 1
            dicionario["LABEL_MISPLACED"] = label_align(img_cru)
        else:
            dicionario['LABEL_WHITE'] = 0
            dicionario["LABEL_MISPLACED"] = label_align(img_cru)

    return np.array(list(dicionario.values()))


img = cv.imread(f'./bottles/train/train_1.jpg')
print(bottle_detection(img))

[0 0 0 0 0 0 0 0]
